In [1]:
import cv2
import json
import requests
import kserve
import numpy as np
import tritonclient.http as httpclient

In [2]:
im = cv2.imread(
    "/home/wut/playground/DQ_Arralyze_MachineLearning/rest_related/mldeployment/test/TestImg.png"
)

In [3]:
api_url = "http://mlserver1.lpkf.com:8000/v2"
response = requests.get(api_url)
response.content

with open("server_info.json", "w") as f:
    # Write the response to a JSON file
    json.dump(response.json(), f)

In [4]:
try:
    # Need to specify large enough concurrency to issue all the
    # inference requests to the server in parallel.
    triton_client = httpclient.InferenceServerClient(
        url="mlserver1.lpkf.com:8000", verbose=False, concurrency=100
    )
except Exception as e:
    print("context creation failed: " + str(e))
model_name = "yolov8_ensemble"

In [5]:
dat = np.asarray(im, dtype=np.float32).reshape(1, 1265, 1268, 3)
inferInput = kserve.InferInput(
    "input_2", [1, 1265, 1268, 3], datatype="FP32", data=dat
)
inferRequest = kserve.InferRequest(infer_inputs=[inferInput], model_name="yolov8_ensemble")
restRequest = inferRequest.to_rest()

In [12]:
r = requests.post(
    "http://mlserver1.lpkf.com:8000/v2/models/yolov8_ensemble/versions/1/infer",
    json=restRequest,
)
# r = requests.post(
#     "http://mlserver1.lpkf.com:8000/v2/models/yolov8_tf/versions/1/infer",
#     json=restRequest,
# )
# r.json()
with open("infer_yolov8_ensemble_with_confidence.json", "w") as f:
    # Write the response to a JSON file
    json.dump(r.json(), f)